In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as pl

In [2]:
pl.style.use('seaborn-dark-palette')

In [3]:
df = pd.read_pickle('./pickleJar/df_main.pkl')

In [4]:
df.head()

,id,etopo2,lat,rrs411,rrs443,rrs489,rrs510,rrs555,rrs670,chlor_a,is_hplc
0,1565,0.0,38.4279,0.001204,0.001686,0.003293,0.004036,0.007479,0.003465,38.19,False
1,1566,0.0,38.3680,0.001062,0.001384,0.002173,0.002499,0.004152,0.001695,35.01,False
2,1567,1.0,38.3074,0.000971,0.001185,0.001843,0.002288,0.004246,0.001612,26.91,False
3,1568,3.0,38.6367,0.001472,0.001741,0.002877,0.003664,0.006982,0.003234,47.96,False
4,1559,1.0,38.3047,0.000905,0.001022,0.001506,0.001903,0.002801,0.001791,23.55,False


In [6]:
df.replace(0, 1e-10, inplace=True)

In [7]:
df.describe()

,id,etopo2,lat,rrs411,rrs443,rrs489,rrs510,rrs555,rrs670,chlor_a
count,4459.000000,4.459000e+03,4459.000000,4293.000000,4456.000000,4422.000000,3435.000000,3255.000000,1.598000e+03,4127.000000
mean,4377.381251,1.312347e+03,1.868658,0.004881,0.004652,0.004590,0.004130,0.003256,1.557001e-03,2.680228
std,2298.272102,1.766435e+03,44.765125,0.003447,0.003002,0.002768,0.003130,0.003536,2.386548e-03,5.758436
min,6.000000,1.000000e-10,-77.035600,0.000051,0.000190,0.000284,0.000261,0.000183,1.000000e-10,0.012000
25%,2028.500000,1.800000e+01,-61.299000,0.002509,0.002617,0.003051,0.002831,0.001588,1.998352e-04,0.233325
50%,5039.000000,2.400000e+02,27.093000,0.003984,0.003899,0.004153,0.003425,0.002071,6.136641e-04,0.764000
75%,6271.500000,2.789500e+03,34.458500,0.006301,0.006076,0.005655,0.004242,0.003141,2.000334e-03,2.150000
max,7831.000000,7.978000e+03,79.690000,0.030600,0.036769,0.063814,0.077740,0.046600,2.769998e-02,77.864800


In [8]:
df.insert(df.shape[1]-2, 'CI',
          df.rrs555 - (df.rrs443
                       + (555-443) / (670-443)
                       * (df.rrs670 - df.rrs443)))

CI criteria for NOMAD (from [Hu*et al.*, 2012](http://dx.doi.org/10.1029/2011jc007395)):
* chl from hplc
* Rrs($\lambda$) > 0.0 sr^{-1}
* Chl > 0.0
* bottom depth > 30.0m
* latitude $\in$ [60$^\circ$S, 60$^\circ$N] 

In [9]:
# Applying above criteria
df.insert(df.columns.get_loc('CI')+1, 'CI_valid', True)

In [10]:
df.loc[((~df.is_hplc) | (df.rrs670<=0) |
       (df.chlor_a<=0) | (df.etopo2<=30) | (df.lat<-60) | 
       (df.lat>60)), 'CI_valid'] = False

In [11]:
df.describe()

,id,etopo2,lat,rrs411,rrs443,rrs489,rrs510,rrs555,rrs670,CI,chlor_a
count,4459.000000,4.459000e+03,4459.000000,4293.000000,4456.000000,4422.000000,3435.000000,3255.000000,1.598000e+03,1163.000000,4127.000000
mean,4377.381251,1.312347e+03,1.868658,0.004881,0.004652,0.004590,0.004130,0.003256,1.557001e-03,0.002061,2.680228
std,2298.272102,1.766435e+03,44.765125,0.003447,0.003002,0.002768,0.003130,0.003536,2.386548e-03,0.003488,5.758436
min,6.000000,1.000000e-10,-77.035600,0.000051,0.000190,0.000284,0.000261,0.000183,1.000000e-10,-0.006578,0.012000
25%,2028.500000,1.800000e+01,-61.299000,0.002509,0.002617,0.003051,0.002831,0.001588,1.998352e-04,0.000274,0.233325
50%,5039.000000,2.400000e+02,27.093000,0.003984,0.003899,0.004153,0.003425,0.002071,6.136641e-04,0.001315,0.764000
75%,6271.500000,2.789500e+03,34.458500,0.006301,0.006076,0.005655,0.004242,0.003141,2.000334e-03,0.003598,2.150000
max,7831.000000,7.978000e+03,79.690000,0.030600,0.036769,0.063814,0.077740,0.046600,2.769998e-02,0.023865,77.864800


In [12]:
ci_thresh = -0.0005
df.insert(df.shape[1]-2, 'within_CI_thresh',
          df.CI<ci_thresh)

In [13]:
df.within_CI_thresh = df.within_CI_thresh.astype('i')

In [14]:
df.head()

,id,etopo2,lat,rrs411,rrs443,rrs489,rrs510,rrs555,rrs670,CI,CI_valid,within_CI_thresh,chlor_a,is_hplc
0,1565,1.000000e-10,38.4279,0.001204,0.001686,0.003293,0.004036,0.007479,0.003465,0.004916,False,0,38.19,False
1,1566,1.000000e-10,38.3680,0.001062,0.001384,0.002173,0.002499,0.004152,0.001695,0.002615,False,0,35.01,False
2,1567,1.000000e+00,38.3074,0.000971,0.001185,0.001843,0.002288,0.004246,0.001612,0.002850,False,0,26.91,False
3,1568,3.000000e+00,38.6367,0.001472,0.001741,0.002877,0.003664,0.006982,0.003234,0.004504,False,0,47.96,False
4,1559,1.000000e+00,38.3047,0.000905,0.001022,0.001506,0.001903,0.002801,0.001791,0.001399,False,0,23.55,False


In [15]:
df.within_CI_thresh.value_counts()

0    4217
1     242
Name: within_CI_thresh, dtype: int64

In [16]:
df.loc[df.CI_valid, 'within_CI_thresh'].value_counts()

0    914
1     50
Name: within_CI_thresh, dtype: int64

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Data columns (total 14 columns):
id                  4459 non-null int64
etopo2              4459 non-null float64
lat                 4459 non-null float64
rrs411              4293 non-null float64
rrs443              4456 non-null float64
rrs489              4422 non-null float64
rrs510              3435 non-null float64
rrs555              3255 non-null float64
rrs670              1598 non-null float64
CI                  1163 non-null float64
CI_valid            4459 non-null bool
within_CI_thresh    4459 non-null int32
chlor_a             4127 non-null float64
is_hplc             4459 non-null bool
dtypes: bool(2), float64(10), int32(1), int64(1)
memory usage: 409.4 KB


In [18]:
df.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1052 entries, 0 to 4458
Data columns (total 14 columns):
id                  1052 non-null int64
etopo2              1052 non-null float64
lat                 1052 non-null float64
rrs411              1052 non-null float64
rrs443              1052 non-null float64
rrs489              1052 non-null float64
rrs510              1052 non-null float64
rrs555              1052 non-null float64
rrs670              1052 non-null float64
CI                  1052 non-null float64
CI_valid            1052 non-null bool
within_CI_thresh    1052 non-null int32
chlor_a             1052 non-null float64
is_hplc             1052 non-null bool
dtypes: bool(2), float64(10), int32(1), int64(1)
memory usage: 104.8 KB


In [19]:
def insert_MBR(df_):
    insert_loc = df_.columns.get_loc('chlor_a')
    blue_cols = ['rrs443', 'rrs489', 'rrs510']
    df_.insert(insert_loc, 'MaxBlue', df_[blue_cols].max(axis=1))
    insert_loc += 1
    df_.insert(insert_loc, 'MaxBlueBand', df_[blue_cols].idxmax(axis=1).str.capitalize())
    insert_loc += 1
    df_.insert(insert_loc,'MaxBlueBandIdx',
               pd.Categorical(df_.MaxBlueBand, categories=[bluecol.capitalize() for bluecol in blue_cols]).codes)
    insert_loc += 1
    df_.insert(insert_loc, 'mxBlue2Gr', df_.MaxBlue / df_.rrs555)
    
    
def log_transform_for_chl_model(df_, merge=False):
    columns_to_log_transform = df_.filter(regex='(rrs|chl|mxBlue2)',
                                             axis=1).columns.tolist()
    df_[columns_to_log_transform].replace(0, 1e-10, inplace=True)
    if merge:
        for col in columns_to_log_transform:
            df_.insert(df_.columns.get_loc(col)+1,
                      f'log_{col}', np.log10(df.loc[:, col]))
        return None
    else:
        # log_transforms
        df_log = pd.DataFrame(np.log10(df_.loc[:,
                                               columns_to_log_transform
                                              ].values),
                              columns=[f'log_{col}' 
                                       for col in columns_to_log_transform])
        return df_log

In [20]:
insert_MBR(df)

In [21]:
log_transform_for_chl_model(df, merge=True)

/Users/erdemk/anaconda3/envs/chl_alg_comparison/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [22]:
df.describe()

,id,etopo2,lat,rrs411,log_rrs411,rrs443,log_rrs443,rrs489,log_rrs489,rrs510,...,rrs670,log_rrs670,CI,within_CI_thresh,MaxBlue,MaxBlueBandIdx,mxBlue2Gr,log_mxBlue2Gr,chlor_a,log_chlor_a
count,4459.000000,4.459000e+03,4459.000000,4293.000000,4293.000000,4456.000000,4456.000000,4422.000000,4422.000000,3435.000000,...,1.598000e+03,1598.000000,1163.000000,4459.000000,4459.000000,4459.000000,3255.000000,3255.000000,4127.000000,4127.000000
mean,4377.381251,1.312347e+03,1.868658,0.004881,-2.419273,0.004652,-2.421040,0.004590,-2.395443,0.004130,...,1.557001e-03,-3.501395,0.002061,0.054272,0.005277,0.843014,2.469670,0.278669,2.680228,-0.112459
std,2298.272102,1.766435e+03,44.765125,0.003447,0.326394,0.003002,0.292499,0.002768,0.224659,0.003130,...,2.386548e-03,1.625059,0.003488,0.226580,0.003407,0.802378,1.900472,0.310451,5.758436,0.687956
min,6.000000,1.000000e-10,-77.035600,0.000051,-4.290672,0.000190,-3.721171,0.000284,-3.546531,0.000261,...,1.000000e-10,-10.000000,-0.006578,0.000000,0.000284,0.000000,0.399575,-0.398402,0.012000,-1.920819
25%,2028.500000,1.800000e+01,-61.299000,0.002509,-2.600448,0.002617,-2.582159,0.003051,-2.515600,0.002831,...,1.998352e-04,-3.699328,0.000274,0.000000,0.003148,0.000000,1.048960,0.020759,0.233325,-0.632039
50%,5039.000000,2.400000e+02,27.093000,0.003984,-2.399659,0.003899,-2.409061,0.004153,-2.381635,0.003425,...,6.136641e-04,-3.212071,0.001315,0.000000,0.004427,1.000000,1.690559,0.228030,0.764000,-0.116907
75%,6271.500000,2.789500e+03,34.458500,0.006301,-2.200602,0.006076,-2.216361,0.005655,-2.247581,0.004242,...,2.000334e-03,-2.698897,0.003598,0.000000,0.006617,2.000000,3.424486,0.534595,2.150000,0.332438
max,7831.000000,7.978000e+03,79.690000,0.030600,-1.514280,0.036769,-1.434521,0.063814,-1.195083,0.077740,...,2.769998e-02,-1.557521,0.023865,1.000000,0.077740,2.000000,10.239706,1.010287,77.864800,1.891341


In [27]:
sb.pairplot(df.loc[df.CI_valid], hue='within_CI_thresh',
           vars=['CI', 'log_mxBlue2Gr', 'log_chlor_a'],
            plot_kws={'alpha':0.75,});